In [57]:
time_start = 20201231
time_end =  20211031
today = 20211130

day_back = 5 # 回溯几天进行计算
day = 30 # 一次提取几天的数,一般默认为30
stride = 10 # 一次学习多少天
feat_list = 

In [72]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm

In [59]:
dataframe_list = DataAPI.get_stock_prices(start_date=20150101, end_date=today).reset_index()
stock = DataAPI.get_universe_series("Investable", start_date=20150101, end_date=today).reset_index()

In [67]:
dataframe_list = pd.merge(left=dataframe_list, right=stock, how='left',
               left_on=['timestamp','ticker'], right_on=['timestamp','ticker'])
dataframe_list = dataframe_list[dataframe_list['tradable'] == 1]
dataframe_list = dataframe_list[['timestamp','ticker','pre_close','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover','chg']]
dataframe_list = dataframe_list.sort_values(['ticker','timestamp'])
dataframe_list['target'] = (dataframe_list['close'].shift(-day_back) - dataframe_list['close']) / dataframe_list['close'] * 100
dataframe_list[dataframe_list['timestamp'] <= time_end]
dataframe_list.set_index(['timestamp','ticker'],inplace = True)
dataframe_list

,timestamp,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,target
0,20150105,000001,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-7.802747
1,20150106,000001,16.02,15.85,16.39,15.55,15.78,15.9408,2166421.40,-1.4981,2.2024,4.3668,-0.24,-6.970849
2,20150107,000001,15.78,15.56,15.83,15.30,15.48,15.4977,1700120.67,-1.9011,1.7283,3.4269,-0.30,-4.328165
3,20150108,000001,15.48,15.50,15.57,14.90,14.96,15.1167,1407714.21,-3.3592,1.4311,2.8375,-0.52,2.606952
4,20150109,000001,14.96,14.90,15.87,14.71,15.08,15.2895,2508500.23,0.8021,2.5501,5.0564,0.12,1.923077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5274775,20211124,688981,54.91,55.16,55.39,54.73,54.77,54.9838,250231.98,-0.2550,1.3374,1.4974,-0.14,NaN
5274776,20211125,688981,54.77,54.77,55.47,54.60,55.00,55.1035,238027.88,0.4199,1.2722,1.4244,0.23,NaN
5274777,20211126,688981,55.00,54.81,55.02,54.15,54.25,54.4330,272853.03,-1.3636,1.4583,1.6328,-0.75,NaN
5274778,20211129,688981,54.25,53.90,54.67,53.71,54.25,54.2378,160234.51,0.0000,0.8564,0.9589,0.00,NaN


In [71]:
dataframe_list.to_csv('/home/wuwenjun/Data/%s_%s.csv' % (time_start,time_end))
print("save to " + '/home/wuwenjun/Data/%s_%s.csv' % (time_start,time_end))

save to /home/wuwenjun/Data/20201231_20211130.csv
